### Create list of all available somatic mutations

In [2]:
import os, glob

FileLst = []
for filename in glob.glob("VCF/*.vcf"):
        FileLst = FileLst + [open(filename, "r")]#open all vcf files 
som_lst = [] #list of somatic mutetions
for file in FileLst:
    line = file.readline()
    while line != "":
        if '#' in line:#skip header rows
            line = file.readline()
            continue
        else:          
            rec = str.split(line,'\t')
            if rec[0]+'_'+rec[1] not in som_lst:
                som_lst += [rec[0]+'_'+rec[1]]                
        line = file.readline()
        line = line.strip('\n')#some vcf files had lines with '\n' only at the end
    file.close()
som_lst.sort()
newfile = open('vcf_matrix.tsv','w')
newfile.write('Position\n')#the title line
for som in som_lst:
    newfile.write(som + '\n')

### Use list of all available somatic mutations to build matrix (1 -> som mut present, 0 -> no som mut)

In [3]:
import os, glob
from shutil import copyfile
copyfile('vcf_matrix.tsv', 'vcf_matrix_processed.tsv')

FileLst = []
for filename in glob.glob("VCF/*.vcf"):
        FileLst = FileLst + [open(filename, "r")]#open all vcf files

for file in FileLst:
    pat_id = str.split(str(file),' ')[1][10:-7]#patient ID extracted from the file name
    #print(pat_id)
    pat_som_lst = [] #list of somatic mutetions for one patient
    line = file.readline()
    while line != "":
        if '#' in line:#skip header rows
            line = file.readline()
            continue
        else:
            rec = str.split(line,'\t')
            pat_som_lst += [rec[0]+'_'+rec[1]]
            line = file.readline()
            line = line.strip('\n')#some vcf files had lines with '\n' only at the end
    newfile = open('vcf_matrix_processed.tsv','r')
    tempfile = open('temp.tsv','w')
    newline = newfile.readline()#read the title line
    newline = newline.rstrip('\n')+'\t'+pat_id+'\n'
    tempfile.write(newline)
    newline = newfile.readline()
    while newline != "":           
        newrec = str.split(newline,'\t')
        if str.strip(newrec[0],'\n') in pat_som_lst:#the given mutation is in a given patient
            newline = newline.rstrip('\n')+'\t1\n'
        else: newline = newline.rstrip('\n')+'\t0\n'
        tempfile.write(newline)
        newline = newfile.readline()
    newfile.close()
    tempfile.close()
    copyfile('temp.tsv', 'vcf_matrix_processed.tsv')               

### Run Kaplan-Meier analysis

In [10]:
from csv import reader
import numpy as np

file_name = 'YourClinicalData.csv'#cohort B
file = open(file_name,'r')
clin_lst = []#create a list of clinical data (patID, value)
file.readline()#skip title line
for rec in reader(file):
    if rec[6].isdigit():#skip NULL values
        clin_lst += [rec[0],int(rec[6])]
    
file_name = 'vcf_matrix_processed.tsv'
file = open(file_name,'r')
for rec in reader(file, delimiter='\t'):
    all_mut_id_lst = []#list of patients ID with somatic mutations
    for i in range(len(rec)):
            all_mut_id_lst += [rec[i]]
    break#stop reading after the first line
for rec in reader(file, delimiter='\t'):#continue reading from the second line
    mut_cnt = 0#count of somatic mutations
    mut_lst = []#list of patients with mutation
    for i in range(1, len(rec)):
        if rec[i] == '1':
            mut_cnt += 1
            mut_lst += [all_mut_id_lst[i]]
    if mut_cnt > 10:#there is more than 10 mutations per patient
        mut_val_lst = []#list of clinical values from patients with mutation 
        nomut_val_lst = []#list of clinical values from patients without mutation
        for pat_id in mut_lst:#loop through patients with mutation
            if pat_id in clin_lst:
                mut_val_lst += [clin_lst[clin_lst.index(pat_id)+1]]
        for j in range(0, len(clin_lst),2):#read every second entry
            if clin_lst[j] not in mut_lst:
                    nomut_val_lst += [clin_lst[j+1]]
        print(rec[0],mut_cnt,int(np.mean(mut_val_lst)),int(np.mean(nomut_val_lst)), \
                int(np.median(mut_val_lst)),int(np.median(nomut_val_lst)))